In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import math, os
# Define some handy network layers

def lrelu(x, rate=0.1):
    return tf.maximum(tf.minimum(x * rate, 0), x)

def conv2d_lrelu(inputs, num_outputs, kernel_size, stride):
    conv = tf.contrib.layers.convolution2d(inputs, num_outputs, kernel_size, stride, 
                                           weights_initializer=tf.contrib.layers.xavier_initializer(),
                                           activation_fn=tf.identity)
    conv = lrelu(conv)
    return conv

def conv2d_t_relu(inputs, num_outputs, kernel_size, stride):
    conv = tf.contrib.layers.convolution2d_transpose(inputs, num_outputs, kernel_size, stride,
                                                     weights_initializer=tf.contrib.layers.xavier_initializer(),
                                                     activation_fn=tf.identity)
    conv = tf.nn.relu(conv)
    return conv

def fc_lrelu(inputs, num_outputs):
    fc = tf.contrib.layers.fully_connected(inputs, num_outputs,
                                           weights_initializer=tf.contrib.layers.xavier_initializer(),
                                           activation_fn=tf.identity)
    fc = lrelu(fc)
    return fc

def fc_relu(inputs, num_outputs):
    fc = tf.contrib.layers.fully_connected(inputs, num_outputs,
                                           weights_initializer=tf.contrib.layers.xavier_initializer(),
                                           activation_fn=tf.identity)
    fc = tf.nn.relu(fc)
    return fc

In [2]:
# Encoder and decoder use the DC-GAN architecture
# Define some handy network layers
'''
tf.reshape(
    tensor,
    shape,
    name=None)
shape=tf.placeholder(tf.float32, shape=[None, 227,227,3] )
我们经常会这样来feed数据,如果在运行的时候想知道None到底是多少,这时候,只能通过tf.shape(x)[0]这种方式来获得.
tensor.get_shape(),返回是一个tuple
'''
#定义结构:
#1：输入是train_x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])，即28*28*1
#None指的batch size的大小，所以可以是任何数。
#2：encoder的layers包括2个卷积层conv1和conv2+1个fc层fc1
#cov1的输入是[None, 28, 28, 1]，根据公式N = (W − F + 2P )/S+1，可以计算得输出是28-4=24/2=12+1=13*13*64=[None,13,13,64]
#cov2的输入是[None,13,13,64]，输出是13-4=9/2=4.5，当尺寸不被整除时，卷积向下取整，池化向上取整，因此=4+1=5*5*128=[None,5,5,128]
#cov2转换shape为3200[None,3200],None个3200的一维vector
#fc1的输入是3200，输出是1024个output，[None,1024]
#3：encoder的输出是fc的20个output[None,20]
#4.decoder的layer有2个fc层和1个卷积层，输出层是反卷积层
#fc1输入是20，输出是1024，[None,1024]
#fc2输入是1024，输出是7*7*128=6272,[None,6272]
#fc2[None,6272] reshape成[None,7,7,128]
#cov1层输入是[None,7,7,128],根据dc-gan的结构,输出是[None,14,14,64]
#5.output层输入是[None,14,14,64]，输出是[None,28,28,64]


def encoder(x, z_dim):
    with tf.variable_scope('encoder'):
        conv1 = conv2d_lrelu(x, 64, 4, 2)#(inputs, num_outputs, kernel_size, stride)
        #None*13*13*64
        conv2 = conv2d_lrelu(conv1, 128, 4, 2)#None*5*5*128
        conv2 = tf.reshape(conv2, [-1, np.prod(conv2.get_shape().as_list()[1:])])    
        '''
        conv2.get_shape().as_list()=[None,5,5,128]
         conv2.get_shape().as_list()[1:]=[5,5,128]
         np.prod(conv2.get_shape().as_list()[1:])=3200
         tf.reshape(-1,3200)=[None,3200]
        '''
        fc1 = fc_lrelu(conv2, 1024)
        return tf.contrib.layers.fully_connected(fc1, z_dim, activation_fn=tf.identity)
def decoder(z, reuse=False):
    with tf.variable_scope('decoder') as vs:
        if reuse:
            vs.reuse_variables()
        fc1 = fc_relu(z, 1024)
        fc2 = fc_relu(fc1, 7*7*128)
        fc2 = tf.reshape(fc2, tf.stack([tf.shape(fc2)[0], 7, 7, 128]))
        conv1 = conv2d_t_relu(fc2, 64, 4, 2)
        output = tf.contrib.layers.convolution2d_transpose(conv1, 1, 4, 2, activation_fn=tf.sigmoid)
        return output
# Build the computation graph for training
z_dim = 20
train_x = tf.placeholder(tf.float32, shape=[None, 24, 82, 1])
train_z = encoder(train_x, z_dim)
train_xr = decoder(train_z)